In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime

In [2]:
warning_list = pd.read_csv("warning_list.csv",encoding="utf-8",index_col=[0])

warning_list["index"] = warning_list["index"].apply(lambda x: int(x))

print(warning_list.dtypes)

index                   int64
label_Chinese          object
label_English          object
function_name          object
function_Chinese       object
function_English       object
group_label_Chinese    object
group_label_English    object
machine_obj_Chinese    object
machine_obj_English    object
dtype: object


In [3]:
warning_list.head(5)

,index,label_Chinese,label_English,function_name,function_Chinese,function_English,group_label_Chinese,group_label_English,machine_obj_Chinese,machine_obj_English
0,0,系统启动,SYSTEM START0UP,cb_gp_powerup_function,功能: 控制柜启动,FUNCTION: CABINET AT START0UP,电控柜公共组件,COMMON CABINET GROUPS,主机电柜,cb_main_maCHine
1,1,警告：储存器内的数据被删除,WARNING: MEMORIZED DATA CANCELLED,cb_gp_powerup_function,功能: 控制柜启动,FUNCTION: CABINET AT START0UP,电控柜公共组件,COMMON CABINET GROUPS,主机电柜,cb_main_maCHine
2,2,与YB511电控柜通讯故障,DOWNSTREAM COMMUNICATION FAULT 0 YB511,cb_main_support_function,控制柜辅助功能,CABINET SUPPORT FUNCTION,电控柜主组件,MAIN CABINET GROUP,主机电柜,cb_main_maCHine
3,3,故障：看门狗监视器Ｋ３,BREAKDOWN: WATCHDOW MONITOR K3,cb_main_support_function,控制柜辅助功能,CABINET SUPPORT FUNCTION,电控柜主组件,MAIN CABINET GROUP,主机电柜,cb_main_maCHine
4,4,防护罩继电器 ２Ｋ２０失效,RELAY FAULT 2K20 GUARD,cb_gp_xx_sg_safety_guard_line_function,0,0,电控柜主组件,MAIN CABINET GROUP,主机电柜,cb_main_maCHine


In [4]:
mydb = mysql.connector.connect(
  host="localhost",
  user="dong12",
  password="ve450",
  database="dong"
)
mycursor = mydb.cursor()

In [5]:
column_name = warning_list.columns
column_name

Index(['index', 'label_Chinese', 'label_English', 'function_name',
       'function_Chinese', 'function_English', 'group_label_Chinese',
       'group_label_English', 'machine_obj_Chinese', 'machine_obj_English'],
      dtype='object')

In [6]:
# can not use index as column name
sql_sentence1 = "CREATE TABLE warning (id INT PRIMARY KEY, "
for i in range(1, len(column_name)):
    temp = column_name[i] + " VARCHAR(255), "
    sql_sentence1 += temp
sql_sentence1

'CREATE TABLE warning (id INT PRIMARY KEY, label_Chinese VARCHAR(255), label_English VARCHAR(255), function_name VARCHAR(255), function_Chinese VARCHAR(255), function_English VARCHAR(255), group_label_Chinese VARCHAR(255), group_label_English VARCHAR(255), machine_obj_Chinese VARCHAR(255), machine_obj_English VARCHAR(255), '

In [7]:
sql_sentence1 += "value INT default 0, time datetime default CURRENT_TIMESTAMP);"
sql_sentence1

'CREATE TABLE warning (id INT PRIMARY KEY, label_Chinese VARCHAR(255), label_English VARCHAR(255), function_name VARCHAR(255), function_Chinese VARCHAR(255), function_English VARCHAR(255), group_label_Chinese VARCHAR(255), group_label_English VARCHAR(255), machine_obj_Chinese VARCHAR(255), machine_obj_English VARCHAR(255), value INT default 0, time datetime default CURRENT_TIMESTAMP);'

In [8]:

mycursor.execute(sql_sentence1)

In [9]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('Persons',)
('customers',)
('warning',)


In [10]:
def searchAll():
    sql_sentence = "SELECT * FROM warning;"
    mycursor.execute(sql_sentence)
    myresult = mycursor.fetchall()
    for x in myresult:
        print(x)

In [11]:
searchAll()

## start inserting value

In [12]:
val = []
for i in range(len(warning_list)):
    row = warning_list.iloc[i]
    row[0] = row[0].item()
    val.append(tuple(row))

/tmp/ipykernel_71836/1333519932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[0] = row[0].item()


In [13]:
print(type(val[0][0]))

<class 'int'>


In [14]:
length = len(val[0])
insert_string = "("
for i in range(length - 1):
    insert_string += "%s, "
insert_string += "%s );"
insert_string

'(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s );'

In [15]:
columns = list(warning_list.columns)
columns[0] = "id"
insert_destination = tuple(columns)
insert_destination = ", ".join(insert_destination)
insert_destination

'id, label_Chinese, label_English, function_name, function_Chinese, function_English, group_label_Chinese, group_label_English, machine_obj_Chinese, machine_obj_English'

In [17]:
sql_sentence2 = "INSERT INTO warning ("
sql_sentence2 += insert_destination + ")"
sql_sentence2 += " VALUES " + insert_string
sql_sentence2

'INSERT INTO warning (id, label_Chinese, label_English, function_name, function_Chinese, function_English, group_label_Chinese, group_label_English, machine_obj_Chinese, machine_obj_English) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s );'

In [18]:

mycursor.executemany(sql_sentence2, val)

mydb.commit()

In [20]:
searchAll()

(0, '系统启动', 'SYSTEM START0UP', 'cb_gp_powerup_function', '功能: 控制柜启动', 'FUNCTION: CABINET AT START0UP', '电控柜公共组件', 'COMMON CABINET GROUPS', '主机电柜', 'cb_main_maCHine', 0, datetime.datetime(2022, 6, 30, 15, 26, 57))
(1, '警告：储存器内的数据被删除', 'WARNING: MEMORIZED DATA CANCELLED', 'cb_gp_powerup_function', '功能: 控制柜启动', 'FUNCTION: CABINET AT START0UP', '电控柜公共组件', 'COMMON CABINET GROUPS', '主机电柜', 'cb_main_maCHine', 0, datetime.datetime(2022, 6, 30, 15, 26, 57))
(2, '与YB511电控柜通讯故障', 'DOWNSTREAM COMMUNICATION FAULT 0 YB511', 'cb_main_support_function', '控制柜辅助功能', 'CABINET SUPPORT FUNCTION', '电控柜主组件', 'MAIN CABINET GROUP', '主机电柜', 'cb_main_maCHine', 0, datetime.datetime(2022, 6, 30, 15, 26, 57))
(3, '故障：看门狗监视器Ｋ３', 'BREAKDOWN: WATCHDOW MONITOR K3', 'cb_main_support_function', '控制柜辅助功能', 'CABINET SUPPORT FUNCTION', '电控柜主组件', 'MAIN CABINET GROUP', '主机电柜', 'cb_main_maCHine', 0, datetime.datetime(2022, 6, 30, 15, 26, 57))
(4, '防护罩继电器  ２Ｋ２０失效', 'RELAY FAULT 2K20  GUARD', 'cb_gp_xx_sg_safety_guard_line_funct